Import the necessary libraries

In [4]:
"""
This file downloads the train, test and validation datasets used for training the CNN.
We have used fiftyone's zoo library to download from the openImages v7.

The entire Open Images V7  database size is 561 GB, It contains ~9 million images that 
around 2 million of them are annotated (Classification, Detection, Segmentation, etc.).
The zoo library supports partial download of the entire dataset which helps a lot with 
our purpose.

* All images have been rescaled so that their largest dimension is at most 1024 pixels.
"""
import fiftyone as fo
import os
import fiftyone.zoo as foz
from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib.image as mpimg # Necessary for reading an image
import matplotlib.patches as patches # Necessary for drawing bounding boxes
import json
import dataHandler as handler

Download the datasets using Fiftyone library. 

We download two datasets, one for training and the other for test

In [2]:
# Download the necessary data and dataset images for training. We have chosen "open-images-v7-object-detection-DS"
# as the name.
dsName = "open-images-v7-object-detection-DS-train"
dsClasses = ["Person"]
dsSplit = "train"
dsLblTypes = ["detections", "classifications"]
nSamples = 1000

if not fo.dataset_exists(dsName):
    dataset = foz.load_zoo_dataset(
        "open-images-v7",
        split = dsSplit,
        label_types = dsLblTypes,
        classes = dsClasses,
        max_samples = nSamples,
        seed = 1,
        shuffle = True,
        dataset_name = dsName,
    )
else:
    datasetTrain = fo.load_dataset(dsName)
    print("Dataset already loaded.")

Dataset already loaded.


In [3]:
# Download the necessary data and dataset images for testing. We have chosen "open-images-v7-object-detection-DS"
# as the name.
dsName = "open-images-v7-object-detection-DS-test"
dsClasses = ["Person"]
dsSplit = "test"
dsLblTypes = ["detections", "classifications"]
nSamples = 1000

if not fo.dataset_exists(dsName):
    dataset = foz.load_zoo_dataset(
        "open-images-v7",
        split = dsSplit,
        label_types = dsLblTypes,
        classes = dsClasses,
        max_samples = nSamples,
        seed = 2,
        shuffle = True,
        dataset_name = dsName,
    )
else:
    datasetTest = fo.load_dataset(dsName)
    print("Dataset already loaded. ")

Dataset already loaded. 


In [ ]:
# To be sure of the downloaded datasets, you can run the code below to see the downloaded datasets
# List the current datasets
fo.list_datasets()

Exporting the datasets to two directories. Each one containing two sub-directories, namely, images and labels

In [ ]:
# Exporting the downloaded datasets to the desired locations.
# Check if the path exists
if not os.path.isdir("./data/images/train/"):
    os.makedirs("./data/images/train/", exist_ok = True)
if not os.path.isdir("./data/labels/train/"):
    os.makedirs("./data/labels/train/", exist_ok = True)

# Test data
datasetTrain.export(
    data_path = "./data/images/train",
    labels_path = "./data/labels/train/labels.json",
    dataset_type = fo.types.FiftyOneImageDetectionDataset,
    classes = dsClasses,
    include_confidence = False
)

# Check if the path exists
if not os.path.isdir("./data/images/test"):
    os.makedirs("./data/images/test", exist_ok = True)
if not os.path.isdir("./data/labels/test/"):
    os.makedirs("./data/labels/test/", exist_ok = True)
# Test data
datasetTest.export(
    data_path = "./data/images/test",
    labels_path = "./data/labels/test/labels.json",
    dataset_type = fo.types.FiftyOneImageDetectionDataset,
    classes = dsClasses,
    include_confidence = False
)

In [12]:
# Deserialize the json file and convert it to text files to make it compatible with YOLO-v1
# Training dataset
file = open("./data/labels/train/labels.json")
js = json.load(file)
for item in js["labels"]:
    txt = ""
    with open(f"./data/labels/train/{item}.txt", 'w') as txtFile:
        for subItem in js["labels"][item]: 
            width = subItem["bounding_box"][2]
            height = subItem["bounding_box"][3]
            centerX = subItem["bounding_box"][0] + width/2
            centerY = subItem["bounding_box"][1] + height/2
            label = subItem["label"]
            txt += f"{label} {centerX} {centerY} {width} {height}\n"
        txtFile.write(txt)
        txtFile.close()
    

file = open("./data/labels/test/labels.json")
js = json.load(file)
for item in js["labels"]:
    txt = ""
    with open(f"./data/labels/test/{item}.txt", 'w') as txtFile:
        for subItem in js["labels"][item]: 
            width = subItem["bounding_box"][2]
            height = subItem["bounding_box"][3]
            centerX = subItem["bounding_box"][0] + width/2
            centerY = subItem["bounding_box"][1] + height/2
            label = subItem["label"]
            txt += f"{label} {centerX} {centerY} {width} {height}\n"
        txtFile.write(txt)
        txtFile.close()
    

View the downloaded datasets. Might not work on the server-side.

In [ ]:
# See the samples
aa = fo.load_dataset("open-images-v7-object-detection-DS-test")
session = fo.launch_app(aa.view())

Process the datasets

In [17]:
# Remove grayscale images 

import os
from PIL import Image
import numpy as np

files = os.listdir("./data/images/test")

for file in files:
    img = np.array(Image.open("./data/images/test/"+file))
    if img.ndim == 2:
        print(f"Removed image {file} and its respective label.")
        os.remove("./data/images/test/"+file)
        os.remove("./data/labels/test/"+file.replace(".jpg",".txt"))

In [16]:
# Remove images with four channels
# Alternatively, you can replace the image as well.

import os
from PIL import Image
import numpy as np

files = os.listdir("./data/images/test")

for file in files:
    img = np.array(Image.open("./data/images/test/"+file))
    if img.shape[2] == 4:
        print(f"Removed image {file} and its respective label.")
        os.remove("./data/images/test/"+file)
        os.remove("./data/labels/test/"+file.replace(".jpg",".txt"))